# Pandas de alto rendimiento: evaluación y consulta

Como ya hemos visto en capítulos anteriores, el poder de la pila PyData se basa en la capacidad de NumPy y Pandas para insertar operaciones básicas en código compilado de nivel inferior a través de una sintaxis intuitiva de nivel superior: los ejemplos son operaciones vectorizadas/difundidas en NumPy y operaciones de tipo agrupación en Pandas.
Si bien estas abstracciones son eficientes y efectivas para muchos casos de uso comunes, a menudo dependen de la creación de objetos intermedios temporales, lo que puede causar una sobrecarga excesiva en el tiempo computacional y el uso de la memoria.

Para solucionar esto, Pandas incluye algunos métodos que le permiten acceder directamente a operaciones de velocidad C sin una costosa asignación de matrices intermedias: `eval` y `query`, que se basan en el [paquete NumExpr](https://github.com/ pydata/numexpr).
En este capítulo lo guiaré a través de su uso y le daré algunas reglas generales sobre cuándo podría pensar en usarlos.

## Consulta y evaluación motivadoras: expresiones compuestas

Hemos visto anteriormente que NumPy y Pandas admiten operaciones vectorizadas rápidas; por ejemplo, al sumar los elementos de dos matrices:

In [1]:
import numpy as np
rng = np.random.default_rng(42)
x = rng.random(1000000)
y = rng.random(1000000)
%timeit x + y

2.21 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Como se analizó en [Cálculo en matrices NumPy: funciones universales] (02.03-Computación-en-arrays-ufuncs.ipynb), esto es mucho más rápido que realizar la suma mediante un bucle o comprensión de Python:

In [2]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)),
                    dtype=x.dtype, count=len(x))

263 ms ± 43.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Pero esta abstracción puede volverse menos eficiente al calcular expresiones compuestas.
Por ejemplo, considere la siguiente expresión:

In [3]:
mask = (x > 0.5) & (y < 0.5)

Debido a que NumPy evalúa cada subexpresión, esto equivale aproximadamente a lo siguiente:

In [4]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

En otras palabras, *cada paso intermedio se asigna explícitamente en la memoria*. Si las matrices `x` e `y` son muy grandes, esto puede generar una memoria significativa y una sobrecarga computacional.
La biblioteca NumExpr le brinda la capacidad de calcular este tipo de expresión compuesta elemento por elemento, sin la necesidad de asignar matrices intermedias completas.
La [documentación de NumExpr](https://github.com/pydata/numexpr) tiene más detalles, pero por el momento es suficiente decir que la biblioteca acepta una *cadena* que proporciona la expresión de estilo NumPy que desee. para calcular:

In [5]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.all(mask == mask_numexpr)

True

El beneficio aquí es que NumExpr evalúa la expresión de una manera que evita matrices temporales siempre que sea posible y, por lo tanto, puede ser mucho más eficiente que NumPy, especialmente para secuencias largas de cálculos en matrices grandes.
Las herramientas `eval` y `query` de Pandas que discutiremos aquí son conceptualmente similares y son esencialmente envoltorios específicos de Pandas de la funcionalidad NumExpr.

## pandas.eval para operaciones eficientes

La función `eval` en Pandas utiliza expresiones de cadena para calcular de manera eficiente operaciones en objetos `DataFrame`.
Por ejemplo, considere los siguientes datos:

In [6]:
import pandas as pd
nrows, ncols = 100000, 100
df1, df2, df3, df4 = (pd.DataFrame(rng.random((nrows, ncols)))
                      for i in range(4))

Para calcular la suma de los cuatro ``DataFrame`` usando el enfoque típico de Pandas, simplemente podemos escribir la suma:

In [7]:
%timeit df1 + df2 + df3 + df4

73.2 ms ± 6.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


El mismo resultado se puede calcular mediante ``pd.eval`` construyendo la expresión como una cadena:

In [8]:
%timeit pd.eval('df1 + df2 + df3 + df4')

34 ms ± 4.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


La versión `eval` de esta expresión es aproximadamente un 50% más rápida (y usa mucha menos memoria), y da el mismo resultado:

In [9]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True

`pd.eval` admite una amplia gama de operaciones.
Para demostrarlos, usaremos los siguientes datos enteros:

In [10]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.integers(0, 1000, (100, 3)))
                           for i in range(5))

#### Operadores aritméticos
`pd.eval` admite todos los operadores aritméticos. Por ejemplo:

In [11]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### Operadores de comparación
`pd.eval` admite todos los operadores de comparación, incluidas las expresiones encadenadas:

In [12]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Operadores bit a bit
`pd.eval` admite los operadores bit a bit `&` y `|`:

In [13]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

Además, admite el uso de los literales `and` y `or` en expresiones booleanas:

In [14]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### Atributos e índices de objetos

`pd.eval` admite el acceso a los atributos del objeto mediante la sintaxis `obj.attr` y a los índices mediante la sintaxis `obj[index]`:

In [15]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Otras operaciones

Otras operaciones, como llamadas a funciones, declaraciones condicionales, bucles y otras construcciones más complicadas actualmente *no* están implementadas en `pd.eval`.
Si desea ejecutar estos tipos de expresiones más complicados, puede utilizar la propia biblioteca NumExpr.

## DataFrame.eval para operaciones por columnas

Así como Pandas tiene una función `pd.eval` de nivel superior, los objetos `DataFrame` tienen un método `eval` que funciona de manera similar.
La ventaja del método "eval" es que se puede hacer referencia a las columnas por su nombre.
Usaremos esta matriz etiquetada como ejemplo:

In [16]:
df = pd.DataFrame(rng.random((1000, 3)), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.850888,0.966709,0.958690
1,0.820126,0.385686,0.061402
2,0.059729,0.831768,0.652259
3,0.244774,0.140322,0.041711
4,0.818205,0.753384,0.578851


Usando `pd.eval` como en la sección anterior, podemos calcular expresiones con las tres columnas como esta:

In [17]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

El método `DataFrame.eval` permite una evaluación mucho más concisa de expresiones con las columnas:

In [18]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

Observe aquí que tratamos *nombres de columnas como variables* dentro de la expresión evaluada, y el resultado es el que desearíamos.

### Asignación en DataFrame.eval

Además de las opciones que acabamos de comentar, `DataFrame.eval` también permite la asignación a cualquier columna.
Usemos el `'DataFrame` de antes, que tiene las columnas `'A'`, `'B'` y `'C'`:

In [19]:
df.head()

,A,B,C
0,0.850888,0.966709,0.958690
1,0.820126,0.385686,0.061402
2,0.059729,0.831768,0.652259
3,0.244774,0.140322,0.041711
4,0.818205,0.753384,0.578851


Podemos usar `df.eval` para crear una nueva columna `'D'` y asignarle un valor calculado a partir de las otras columnas:

In [20]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.850888,0.966709,0.958690,1.895916
1,0.820126,0.385686,0.061402,19.638139
2,0.059729,0.831768,0.652259,1.366782
3,0.244774,0.140322,0.041711,9.232370
4,0.818205,0.753384,0.578851,2.715013


De la misma forma se puede modificar cualquier columna existente:

In [21]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.850888,0.966709,0.958690,-0.120812
1,0.820126,0.385686,0.061402,7.075399
2,0.059729,0.831768,0.652259,-1.183638
3,0.244774,0.140322,0.041711,2.504142
4,0.818205,0.753384,0.578851,0.111982


### Variables locales en DataFrame.eval

El método `DataFrame.eval` admite una sintaxis adicional que le permite trabajar con variables locales de Python.
Considere lo siguiente:

In [22]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

El carácter `@` aquí marca un *nombre de variable* en lugar de un *nombre de columna*, y le permite evaluar eficientemente expresiones que involucran los dos "espacios de nombres": el espacio de nombres de las columnas y el espacio de nombres de los objetos de Python.
Tenga en cuenta que este carácter `@` solo es compatible con el *método* `DataFrame.eval`, no con la *función* `pandas.eval`, porque la función `pandas.eval` solo tiene acceso a uno (Python) espacio de nombres.

## El método DataFrame.query

El `DataFrame` tiene otro método basado en cadenas evaluadas, llamado `query`.
Considere lo siguiente:

In [23]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

Al igual que con el ejemplo utilizado en nuestra discusión sobre `DataFrame.eval`, esta es una expresión que involucra columnas de `DataFrame`.
Sin embargo, no se puede expresar utilizando la sintaxis `DataFrame.eval`.
En su lugar, para este tipo de operación de filtrado, puede utilizar el método "consulta":

In [24]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

Además de ser un cálculo más eficiente, en comparación con la expresión de enmascaramiento, es mucho más fácil de leer y comprender.
Tenga en cuenta que el método `query` también acepta el indicador `@` para marcar variables locales:

In [25]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## Rendimiento: cuándo utilizar estas funciones

Al considerar si se debe utilizar `eval` y `query`, hay dos consideraciones: *tiempo de cálculo* y *uso de memoria*.
El uso de la memoria es el aspecto más predecible. Como ya se mencionó, cada expresión compuesta que involucre matrices NumPy o ``DataFrame`` de Pandas dará como resultado la creación implícita de matrices temporales. Por ejemplo, esto:

In [26]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

es aproximadamente equivalente a esto:

In [27]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

Si el tamaño del ``DataFrame`` temporal es significativo en comparación con la memoria disponible del sistema (normalmente varios gigabytes), entonces es una buena idea utilizar una expresión `eval` o `query`.
Puede verificar el tamaño aproximado de su matriz en bytes usando esto:

In [28]:
df.values.nbytes

32000

En cuanto al rendimiento, `eval` puede ser más rápido incluso cuando no estás maximizando la memoria del sistema.
El problema es cómo se comparan sus objetos temporales con el tamaño de la caché de la CPU L1 o L2 de su sistema (normalmente unos pocos megabytes); si son mucho más grandes, entonces "eval" puede evitar algún movimiento potencialmente lento de valores entre las diferentes memorias caché.
En la práctica, encuentro que la diferencia en el tiempo de cálculo entre los métodos tradicionales y el método `eval`/`query` generalmente no es significativa; en todo caso, ¡el método tradicional es más rápido para matrices más pequeñas!
El beneficio de `eval`/`query` está principalmente en la memoria guardada y en la sintaxis a veces más limpia que ofrecen.

Hemos cubierto la mayoría de los detalles de "eval" y "query" aquí; Para obtener más información sobre estos, puede consultar la documentación de Pandas.
En particular, se pueden especificar diferentes analizadores y motores para ejecutar estas consultas; Para obtener detalles sobre esto, consulte la discusión en la sección ["Mejora del rendimiento"](https://pandas.pydata.org/pandas-docs/dev/user_guide/enhancingperf.html) de la documentación.